In [13]:
config = {
    'host':'host',
    'user':'user',
    'password':'password',
    'database':'database'
}

In [14]:
import mysql.connector
from mysql.connector import errorcode
# Construct connection string
try:
    conn = mysql.connector.connect(**config)
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor()

Connection established


In [5]:
# productsテーブルの作成
cursor.execute("""
CREATE TABLE products (
    id INT AUTO_INCREMENT PRIMARY KEY,
    code VARCHAR(13) NOT NULL UNIQUE,
    name VARCHAR(50),
    price INT NOT NULL
);
""")
print("productsテーブルを作成しました。")

# tradesテーブルの作成
cursor.execute("""
CREATE TABLE trades (
    id INT AUTO_INCREMENT PRIMARY KEY,
    datetime DATETIME DEFAULT CURRENT_TIMESTAMP,
    emp_cd VARCHAR(10) NOT NULL,
    store_cd VARCHAR(5) NOT NULL,
    pos_cd VARCHAR(3) NOT NULL,
    total_amt INT,
    ttl_amt_ex_tax INT
);
""")
print("tradesテーブルを作成しました。")

# taxsテーブルの作成
cursor.execute("""
CREATE TABLE taxs (
    trd_id INT AUTO_INCREMENT PRIMARY KEY,
    code VARCHAR(2) UNIQUE,
    name VARCHAR(20),
    percent DECIMAL(5, 2)
);
""")
print("taxsテーブルを作成しました。")

# trade_detailsテーブルの作成
cursor.execute("""
CREATE TABLE trade_details (
    trd_id INT NOT NULL,
    dtl_id INT AUTO_INCREMENT PRIMARY KEY,
    prd_id INT NOT NULL,
    prd_code VARCHAR(13) NOT NULL,
    prd_name VARCHAR(50) NOT NULL,
    prd_price INT NOT NULL,
    tax_cd VARCHAR(2),
    FOREIGN KEY (trd_id) REFERENCES trades(trd_id),
    FOREIGN KEY (prd_id) REFERENCES products(id),
    FOREIGN KEY (tax_cd) REFERENCES taxs(code)
);
""")
print("trade_detailsテーブルを作成しました。")

productsテーブルを作成しました。
tradesテーブルを作成しました。
taxsテーブルを作成しました。


DatabaseError: 1215 (HY000): Cannot add foreign key constraint

In [7]:
# trade_detailsテーブルの作成
cursor.execute("""
CREATE TABLE trade_details (
    trd_id INT NOT NULL,
    dtl_id INT AUTO_INCREMENT PRIMARY KEY,
    prd_id INT NOT NULL,
    prd_code VARCHAR(13) NOT NULL,
    prd_name VARCHAR(50) NOT NULL,
    prd_price INT NOT NULL,
    tax_cd VARCHAR(2),
    FOREIGN KEY (trd_id) REFERENCES trades(id),
    FOREIGN KEY (prd_id) REFERENCES products(id),
    FOREIGN KEY (tax_cd) REFERENCES taxs(code)
);
""")
print("trade_detailsテーブルを作成しました。")

trade_detailsテーブルを作成しました。


In [16]:
# productsテーブルから全てのデータを削除
cursor.execute("DELETE FROM products")
conn.commit()
print("productsテーブルから全てのデータを削除しました。")

productsテーブルから全てのデータを削除しました。


In [17]:
products_data = [
    ('49716351', 'ハウス バジル 瓶 4g', 120),
    ('9784095018553', 'オールカラー 学習漢字新辞典 第2版', 1595),
    ('4513574029606', '麗白 ハトムギ ボディミルク 400ml', 153),
    ('4987286414263', 'ベビーワセリン 100g', 400)
]

for product in products_data:
    cursor.execute(
        "INSERT INTO products (code, name, price) VALUES (%s, %s, %s)",
        product
    )

conn.commit()

In [18]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print("データベース内のテーブル一覧:")
for table in tables:
    print(table[0])

データベース内のテーブル一覧:
products
taxs
trade_details
trades


In [23]:
cursor.execute("DESCRIBE taxs")
columns = cursor.fetchall()
print("\n'products'テーブルのカラム情報:")
for column in columns:
    print(column)


'products'テーブルのカラム情報:
('trd_id', 'int(11)', 'NO', 'PRI', None, 'auto_increment')
('code', 'varchar(2)', 'YES', 'UNI', None, '')
('name', 'varchar(20)', 'YES', '', None, '')
('percent', 'decimal(5,2)', 'YES', '', None, '')


In [22]:
cursor.execute("SELECT * FROM products")
records = cursor.fetchall()
print("\n'products'テーブルのレコード:")
for record in records:
    print(record)


'products'テーブルのレコード:
(5, '49716351', 'ハウス バジル 瓶 4g', 120)
(6, '9784095018553', 'オールカラー 学習漢字新辞典 第2版', 1595)
(7, '4513574029606', '麗白 ハトムギ ボディミルク 400ml', 153)
(8, '4987286414263', 'ベビーワセリン 100g', 400)


In [25]:
cursor.execute("ALTER TABLE taxs CHANGE COLUMN trd_id id INT AUTO_INCREMENT")
conn.commit()
print("taxsテーブルのカラム名を 'trd_id' から 'id' に変更しました。")

taxsテーブルのカラム名を 'trd_id' から 'id' に変更しました。


In [33]:
cursor.execute("DESCRIBE trade_details")
columns = cursor.fetchall()
print("\n'trade_details'テーブルのカラム情報:")
for column in columns:
    print(column)


'trade_details'テーブルのカラム情報:
('trd_id', 'int(11)', 'NO', 'MUL', None, '')
('dtl_id', 'int(11)', 'NO', 'PRI', None, 'auto_increment')
('prd_id', 'int(11)', 'NO', 'MUL', None, '')
('prd_code', 'varchar(13)', 'NO', '', None, '')
('prd_name', 'varchar(50)', 'NO', '', None, '')
('prd_price', 'int(11)', 'NO', '', None, '')
('tax_cd', 'varchar(2)', 'YES', 'MUL', None, '')


In [27]:
# 追加したいレコードの値
code = '10'  # VARCHAR型なので文字列として扱います
name = '10%'  # 同上
percent = 10.00  # DECIMAL型

# INSERT INTO ステートメントを実行
cursor.execute(
    "INSERT INTO taxs (code, name, percent) VALUES (%s, %s, %s)",
    (code, name, percent)
)

# 変更をコミット
conn.commit()

print("taxsテーブルに新しいレコードを追加しました。")

taxsテーブルに新しいレコードを追加しました。


In [28]:
cursor.execute("SELECT * FROM taxs")
records = cursor.fetchall()
print("\n'taxs'テーブルのレコード:")
for record in records:
    print(record)


'taxs'テーブルのレコード:
(1, '10', '10%', Decimal('10.00'))


In [30]:
cursor.execute("SELECT * FROM trades")
records = cursor.fetchall()
print("\n'trades'テーブルのレコード:")
for record in records:
    print(record)


'trades'テーブルのレコード:
(1, datetime.datetime(2024, 2, 20, 22, 45, 49), '', '30', '90', 0, None)


In [36]:
cursor.execute("SELECT * FROM trades")
records = cursor.fetchall()
print("\n'trades'テーブルのレコード:")
for record in records:
    print(record)


'trades'テーブルのレコード:
(1, datetime.datetime(2024, 2, 20, 22, 45, 49), '', '30', '90', 0, None)
